In [1]:
###PRE-BERT DATA Model 

In [2]:
!pip3 install plotly

     |████████████████████████████████| 27.7 MB 9.9 MB/s eta 0:00:01


In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot

import numpy as np
from sklearn.model_selection import train_test_split
#Transforming
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

#cluster and Eval
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
from sklearn.manifold import TSNE
from sklearn import decomposition

#Visualization
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.svm import LinearSVC

In [2]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_excel("loan_data_filtered_important.xlsx")
df.head(5)

,int_rate,installment_percent_of_income,revol_bal,loan_status,annual_inc,pub_rec,installment,funded_amnt_inv,revol_util,sub_grade,...,TX,KY,LA,VA,MI,OK,NV,AL,CA,MS
0,0.1065,0.006786,13648,1,24000.0,0,162.87,4975.0,0.837,6,...,0,0,0,0,0,0,0,0,0,0
1,0.1527,0.001994,1687,0,30000.0,0,59.83,2500.0,0.094,13,...,0,0,0,0,0,0,0,0,0,0
2,0.1349,0.006897,5598,1,49200.0,0,339.31,10000.0,0.210,10,...,0,0,0,0,0,0,0,0,1,0
3,0.1596,0.003618,17726,1,47004.0,0,170.08,7000.0,0.856,14,...,0,0,0,0,0,0,0,0,0,0
4,0.1864,0.002280,8221,1,48000.0,0,109.43,3000.0,0.875,20,...,0,0,0,0,0,0,0,0,1,0


In [4]:
X = df.drop(['loan_status'],axis = 1)
y = df['loan_status']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

X_train.shape, X_test.shape
cols = X_train.columns

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns=[cols])
X_test = pd.DataFrame(X_test, columns=[cols])

# 1. SVM

In [5]:
from sklearn import metrics
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
svc=SVC() 
svc.fit(X_train,y_train)
y_pred_svc=svc.predict(X_test)

print('Model accuracy score : {0:0.4f}'. format(accuracy_score(y_test, y_pred_svc)))
print('Confusion metrics :\n', metrics.confusion_matrix(y_test, y_pred_svc,labels = [1,0]))
print(metrics.classification_report(y_test,y_pred_svc))

Model accuracy score : 0.8544
Confusion metrics :
 [[6515    0]
 [1110    0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1110
           1       0.85      1.00      0.92      6515

    accuracy                           0.85      7625
   macro avg       0.43      0.50      0.46      7625
weighted avg       0.73      0.85      0.79      7625



## Parameters tuning

In [6]:
from sklearn.model_selection import GridSearchCV

svc = SVC()

scorer = make_scorer(f1_score)
param_grid = {'C': [0.1,1, 10, 100]}
svc_turn_cv = GridSearchCV(estimator=svc,
                           param_grid=param_grid,
                           scoring=scorer,
                           cv=10)
svc_turn_cv.fit(X_train, y_train)



# Print hyperparameter
print("Tuned hyperparameter parameters: {}".format(svc_turn_cv.best_params_)) 
print("Best cross-validation score: {}".format(svc_turn_cv.best_score_))

Tuned hyperparameter parameters: {'C': 0.1}
Best cross-validation score: 0.9221727483866229


In [7]:
from sklearn.model_selection import GridSearchCV

svc = SVC()

scorer = make_scorer(f1_score)
param_grid = {'gamma': [0.01,0.001,0.0001]}
svc_turn_cv = GridSearchCV(estimator=svc,
                           param_grid=param_grid,
                           scoring=scorer,
                           cv=10)
svc_turn_cv.fit(X_train, y_train)



# Print hyperparameter
print("Tuned hyperparameter parameters: {}".format(svc_turn_cv.best_params_)) 
print("Best cross-validation score: {}".format(svc_turn_cv.best_score_))

Tuned hyperparameter parameters: {'gamma': 0.01}
Best cross-validation score: 0.9221074238293262


In [8]:
from sklearn.model_selection import GridSearchCV

svc = SVC()

scorer = make_scorer(f1_score)
param_grid = {'kernel': ['rbf', 'poly', 'sigmoid']}
svc_turn_cv = GridSearchCV(estimator=svc,
                           param_grid=param_grid,
                           scoring=scorer,
                           cv=10)
svc_turn_cv.fit(X_train, y_train)



# Print hyperparameter
print("Tuned hyperparameter parameters: {}".format(svc_turn_cv.best_params_)) 
print("Best cross-validation score: {}".format(svc_turn_cv.best_score_))

Tuned hyperparameter parameters: {'kernel': 'rbf'}
Best cross-validation score: 0.9221400861079745


## Final SVM model

In [9]:
from sklearn import metrics
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
svc=SVC(C = 0.1, gamma=0.01,kernel='rbf') 
svc.fit(X_train,y_train)
y_pred_svc=svc.predict(X_test)

print('Model accuracy score : {0:0.4f}'. format(accuracy_score(y_test, y_pred_svc)))
print('Confusion metrics :\n', metrics.confusion_matrix(y_test, y_pred_svc,labels = [1,0]))
print(metrics.classification_report(y_test,y_pred_svc))

Model accuracy score : 0.8544
Confusion metrics :
 [[6515    0]
 [1110    0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1110
           1       0.85      1.00      0.92      6515

    accuracy                           0.85      7625
   macro avg       0.43      0.50      0.46      7625
weighted avg       0.73      0.85      0.79      7625



# 2. Logistics Regression

In [10]:
lg = LogisticRegression(random_state=0)
lg.fit(X_train,y_train)
y_pred_lg = lg.predict(X_test)

print(metrics.classification_report(y_test,y_pred_lg))
print('Confusion metrics :\n', metrics.confusion_matrix(y_test, y_pred_lg,labels = [1,0]))
print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(y_test, y_pred_lg)))

              precision    recall  f1-score   support

           0       0.45      0.02      0.03      1110
           1       0.86      1.00      0.92      6515

    accuracy                           0.85      7625
   macro avg       0.66      0.51      0.48      7625
weighted avg       0.80      0.85      0.79      7625

Confusion metrics :
 [[6491   24]
 [1090   20]]
Model accuracy score with default hyperparameters: 0.8539


In [14]:
print(lg)

LogisticRegression(random_state=0)


In [15]:
# import sklearn

# sklearn.metrics.SCORERS.keys()

## Parameters tuning

In [11]:
from sklearn.model_selection import GridSearchCV
lg = LogisticRegression()
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10,100],
              'solver': ['liblinear', 'lbfgs', 'sag', 'saga','newton-cg'],
              'penalty':['l1','l2','elasticnet','none']}

from sklearn import metrics
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score

# scorer = make_scorer(f1_score, pos_label = 0)
scorer = make_scorer(f1_score)
lg_tuning = GridSearchCV(estimator=lg,
                         param_grid=param_grid,
                         scoring=scorer,
                         cv=10)
lg_tuning.fit(X_train, y_train)



# Print hyperparameter
print("Tuned hyperparameter parameters: {}".format(lg_tuning.best_params_)) 
print("Best cross-validation score: {}".format(lg_tuning.best_score_))

Tuned hyperparameter parameters: {'C': 0.001, 'penalty': 'l1', 'solver': 'liblinear'}
Best cross-validation score: 0.9221727483866229


### Final LG mdoel

In [12]:
from sklearn import metrics
lg = LogisticRegression(C = 0.001, solver = 'liblinear', penalty = 'l1')
lg.fit(X_train,y_train)
y_pred_lg = lg.predict(X_test)

print('Model accuracy score : {0:0.4f}'. format(accuracy_score(y_test, y_pred_lg)))
print('Confusion metrics :\n', metrics.confusion_matrix(y_test, y_pred_lg,labels = [1,0]))
print(metrics.classification_report(y_test,y_pred_lg))

Model accuracy score : 0.8544
Confusion metrics :
 [[6515    0]
 [1110    0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1110
           1       0.85      1.00      0.92      6515

    accuracy                           0.85      7625
   macro avg       0.43      0.50      0.46      7625
weighted avg       0.73      0.85      0.79      7625



# 3. Decision tree

In [13]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
y_pred_dt = dt.predict(X_test)

print('Model accuracy score : {0:0.4f}'. format(accuracy_score(y_test, y_pred_dt)))
print('Confusion metrics :\n', metrics.confusion_matrix(y_test, y_pred_dt))
print(metrics.classification_report(y_test,y_pred_dt))

Model accuracy score : 0.7557
Confusion metrics :
 [[ 250  860]
 [1003 5512]]
              precision    recall  f1-score   support

           0       0.20      0.23      0.21      1110
           1       0.87      0.85      0.86      6515

    accuracy                           0.76      7625
   macro avg       0.53      0.54      0.53      7625
weighted avg       0.77      0.76      0.76      7625



In [14]:
print(dt)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


## Parameter tuning

In [15]:
from sklearn.model_selection import GridSearchCV
dt = DecisionTreeClassifier()
param_grid = {'criterion' : ['gini', 'entropy'],
              'max_depth' : [2,4,6,8,10,12,14]}

from sklearn import metrics
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score


scorer = make_scorer(f1_score)
dt_tuning = GridSearchCV(estimator=dt,
                         param_grid=param_grid,
                         scoring=scorer,
                         cv=10)
dt_tuning.fit(X_train, y_train)



# Print hyperparameter
print("Tuned hyperparameter parameters: {}".format(dt_tuning.best_params_)) 
print("Best cross-validation score: {}".format(dt_tuning.best_score_))



Tuned hyperparameter parameters: {'criterion': 'gini', 'max_depth': 2}
Best cross-validation score: 0.9221727483866229


## Final DT model

In [16]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion='gini',max_depth=2)
dt.fit(X_train,y_train)
y_pred_dt = dt.predict(X_test)

print('Model accuracy score : {0:0.4f}'. format(accuracy_score(y_test, y_pred_dt)))
print('Confusion metrics :\n', metrics.confusion_matrix(y_test, y_pred_dt))
print(metrics.classification_report(y_test,y_pred_dt))

Model accuracy score : 0.8544
Confusion metrics :
 [[   0 1110]
 [   0 6515]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1110
           1       0.85      1.00      0.92      6515

    accuracy                           0.85      7625
   macro avg       0.43      0.50      0.46      7625
weighted avg       0.73      0.85      0.79      7625

